In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from processing import *

c:\Work_life\HSE_and_study\Intern\Avito\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "facebook/mbart-large-50"
checkpoint_dir = "mbart-space-fix\checkpoint-1300"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [3]:
def predict_spaces(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", max_length=32, truncation=True, padding=True).to(model.device)
    outputs = model.generate(**inputs, max_length=32)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [4]:
df_test = pd.read_csv('train_model_data\wiki_dataset.csv', sep=';')
df_train, df_test = train_test_split(df_test, test_size=0.1, shuffle=True)
df_test.sample(10)

,text_no_spaces,text_restored
5474,негоприходится,него приходится
4703,долларовДругая,долларов Другая
10087,Casio22,Casio 22
11645,принтерTCL65,принтер TCL 65
9816,1957годуВэтой,1957 году В этой
1398,1987годуMicrosoft,1987 году Microsoft
3746,Boonstra20012011,Boonstra 20012011
4404,Компанияпересталапринимать,Компания перестала принимать
1224,сыннаследует,сын наследует
8627,основанныхнадисках,основанных на дисках


In [5]:
restorer = SpaceRestorer()

In [6]:
df_test['text_processed'] = df_test['text_no_spaces'].apply(restorer.restore_spaces)

In [7]:
df_test.sample(5)

,text_no_spaces,text_restored,text_processed
1091,УкраиныВиюне2022,Украины В июне 2022,Украины Виюне 2022
6851,вмире,в мире,вмире
4300,MobileиHMDGlobal,Mobile и HMD Global,Mobile и HMDGlobal
3164,перестановкитопменеджеров,перестановки топменеджеров,перестановкитопменеджеров
11955,Lacoste7диван,Lacoste 7 диван,Lacoste 7 диван


In [8]:
test_text = "придуманбрат Луиджи Позже"
prediction = predict_spaces(test_text)
print(f"Input: {test_text}")
print(f"Output: {prediction}")

Input: придуманбрат Луиджи Позже
Output: придуман брат Луиджи Позже


In [9]:
df_test['pred'] = df_test['text_processed'].apply(predict_spaces)

In [10]:
df_test.sample(10)

,text_no_spaces,text_restored,text_processed,pred
11291,"Lego,12Pro","Lego, 12 Pro","Lego, 12 Pro","Lego, 12 Pro"
2567,доконцагода,до конца года,доконцагода,до конца года
12128,пицца50,пицца 50,пицца 50,пицца 50
11138,AcerOne,AcerOne,Acer One,Acer One
196,акцииторгуютсянабирже,акции торгуются на бирже,акцииторгуютсянабирже,акции торгуются на бирже
11087,Москва14,Москва 14,Москва 14,Москва 14
1466,условияхтруда,условиях труда,условияхтруда,условиях труда
10808,курткаMotorola22,куртка Motorola 22,куртка Motorola 22,куртка Motorola 22
7874,Архивировано2,Архивировано 2,Архивировано 2,Архивировано 2
9320,1995June,1995 June,1995 June,1995 June


In [12]:
df_test['real_index'] = df_test['text_restored'].apply(find_space_indices)
df_test['pred_index'] = df_test['pred'].apply(find_space_indices)

In [13]:
df_test

,text_no_spaces,text_restored,text_processed,pred,real_index,pred_index
5797,былозаключеносоглашение,было заключено соглашение,былозаключеносоглашение,было заключено соглашение,"[4, 14]","[4, 14]"
468,электромобили9Зарубежныепродажи,электромобили 9 Зарубежные продажи,электромобили 9 Зарубежныепродажи,электромобили 9 Зарубежные продажи,"[13, 15, 26]","[13, 15, 26]"
6816,ГовардаЛинкольнав,Говарда Линкольна в,Говарда Линкольнав,Говарда Линкольна в,"[7, 17]","[7, 17]"
5997,PS1выпущеннаяв,PS1 выпущенная в,PS 1 выпущеннаяв,PS 1 выпущенная в,"[3, 14]","[2, 4, 15]"
8960,2000хгодовначала,2000х годов начала,2000 хгодовначала,2000 х годов начала,"[5, 11]","[4, 6, 12]"
...,...,...,...,...,...,...
12186,креслоiPhone28,кресло iPhone 28,кресло i Phone 28,кресло i Phone 28,"[6, 13]","[6, 8, 14]"
1662,XHTMLтегиМетасуданс,XHTMLтеги Мета суданс,XHTML теги Метасуданс,XHTML теги Мета суданс,"[9, 14]","[5, 10, 15]"
8750,вавтомобильной,в автомобильной,вавтомобильной,в автомобильной,[1],[1]
4648,игркак,игр как,игркак,игр как,[3],[3]


In [ ]:
from typing import List
import numpy as np
import pandas as pd

def compute_f1(real_indices: List[int], pred_indices: List[int]) -> float:
    if not pred_indices or not real_indices:
        return 0.0
    true_positives = len(set(real_indices) & set(pred_indices))
    precision = true_positives / len(pred_indices) if pred_indices else 0.0
    recall = true_positives / len(real_indices) if real_indices else 0.0
    if precision + recall == 0:
        return 0.0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def evaluate_f1_scores(data: pd.DataFrame):
    f1_scores = []
    for _, example in data.iterrows():  
        real_indices = example["real_index"]
        pred_indices = example["pred_index"]
        f1 = compute_f1(real_indices, pred_indices)
        f1_scores.append(f1)
        # print(f"Real indices: {real_indices}, Pred indices: {pred_indices}, F1: {f1:.4f}")
    mean_f1 = np.mean(f1_scores) if f1_scores else 0.0
    return mean_f1



evaluate_f1_scores(df_test)

np.float64(0.9369865689865691)